In [26]:
import pandas as pd
from thefuzz import fuzz
from helpers import classification as pf
from helpers import stats as bf
from services import BetExplorerService, MySQLService
import os
from joblib import load
from termcolor import colored

from helpers.file import load_from_file
from helpers.options import filtered_cols, selected_stats, strategy

In [38]:
bankroll = 900
season = 2024
league = "laliga"
country = "spain"

In [39]:
def load_saved_predictor_utils(league):
    predictor_utils = load(f"../dist/betting/{league}.joblib")

    return predictor_utils

model_data = load_saved_predictor_utils(league)

pipeline = model_data["pipeline"]
best_model_name = model_data["best_model_name"]
features = model_data["filtered_cols"]
min_odds = model_data["min_odds"]
min_games_played = model_data["min_games_played"]
min_games_played_at = model_data["min_games_played_at"]
strategy = model_data["strategy"]
default_bankroll_pct = model_data["default_bankroll_pct"]
default_value = model_data["default_value"]

print(f"Model {best_model_name}, features, and min odds have been loaded successfully.")
print(f"Loaded features: {features}")
print(f"Loaded min odds: {min_odds}")
print(f"Loaded mininum games played: {min_games_played}")
print(f"Loaded mininum games played at: {min_games_played_at}")
print(f"Strategy: {strategy}")
print(f"Default bankroll pct: {default_bankroll_pct}")
print(f"Default value: {default_value}")

Model voting_classifier, features, and min odds have been loaded successfully.
Loaded features: ['home_points_pct', 'home_win_pct', 'home_draw_pct', 'home_loss_pct', 'home_points_pct_last_games', 'home_win_pct_last_games', 'home_draw_pct_last_games', 'home_loss_pct_last_games', 'home_home_win_pct', 'home_home_draw_pct', 'home_home_loss_pct', 'home_team_score', 'home_opp_score', 'home_home_team_score', 'home_home_opp_score', 'away_points_pct', 'away_win_pct', 'away_draw_pct', 'away_loss_pct', 'away_points_pct_last_games', 'away_win_pct_last_games', 'away_draw_pct_last_games', 'away_loss_pct_last_games', 'away_away_win_pct', 'away_away_draw_pct', 'away_away_loss_pct', 'away_team_score', 'away_opp_score', 'away_away_opp_score', 'away_away_team_score', 'home_odds', 'away_odds', 'draw_odds', 'home_elo', 'away_elo']
Loaded min odds: 2.2
Loaded mininum games played: 10
Loaded mininum games played at: 5
Strategy: default
Default bankroll pct: 0.05
Default value: 1


In [40]:
pipeline

Pipeline(steps=[('model',
                 VotingClassifier(estimators=[('random_forest_default',
                                               RandomForestClassifier(random_state=0)),
                                              ('gradient_boosting',
                                               GradientBoostingClassifier(random_state=0))],
                                  voting='soft'))])

In [41]:
mysql = MySQLService()

season_games, teams_elo = bf.initialize_matches(league, season)

teams_query = f"SELECT DISTINCT(home_team) as team FROM matches WHERE season = {season} and league = '{league}'"

teams = mysql.execute_query(teams_query)

Generating teams ELOs...


100%|██████████| 4678/4678 [00:00<00:00, 8791.65it/s]

Successfully generated teams ELOs.


In [42]:
display(season_games)

,season,league,date,week,home_team,home_xg,home_score,away_score,away_xg,away_team,home_odds,away_odds,draw_odds,result,home_elo,away_elo
4560,2024,laliga,2024-08-15,1,Athletic Club,0.3,1,1,0.8,Getafe,1.60,6.91,3.55,D,1626.91,1465.44
4561,2024,laliga,2024-08-15,1,Betis,1.4,1,1,1.6,Girona,2.32,3.12,3.36,D,1590.50,1669.55
4562,2024,laliga,2024-08-16,1,Celta Vigo,0.8,2,1,1.6,Alavés,2.10,3.85,3.19,H,1498.33,1481.14
4563,2024,laliga,2024-08-16,1,Las Palmas,1.4,2,2,1.8,Sevilla,2.61,2.83,3.17,D,1400.63,1518.46
4564,2024,laliga,2024-08-17,1,Osasuna,1.7,1,1,1.0,Leganés,1.86,4.91,3.20,D,1499.29,1485.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4673,2024,laliga,2024-11-03,12,Athletic Club,4.0,1,1,0.9,Betis,1.86,4.31,3.55,D,1630.90,1598.44
4674,2024,laliga,2024-11-03,12,Atlético Madrid,1.6,2,0,0.0,Las Palmas,1.33,9.01,5.22,H,1710.75,1404.51
4675,2024,laliga,2024-11-03,12,Barcelona,1.8,3,1,0.9,Espanyol,1.15,16.36,8.44,H,1833.33,1435.13
4676,2024,laliga,2024-11-03,12,Sevilla,1.1,0,2,1.4,Real Sociedad,3.35,2.32,3.12,A,1526.53,1584.82


In [43]:
# Getting odds for next games

bet_explorer = BetExplorerService(country, league)

next_games = bet_explorer.get_next_games()

print(next_games)

Exception when fetching a game: too many values to unpack (expected 9)
[{'home_team': 'Rayo Vallecano', 'away_team': 'Las Palmas', 'home_odds': 1.88, 'draw_odds': 3.41, 'away_odds': 4.53, 'date': datetime.datetime(2024, 11, 8, 12, 50, 52, 35538)}, {'home_team': 'Real Madrid', 'away_team': 'Osasuna', 'home_odds': 1.2, 'draw_odds': 7.11, 'away_odds': 13.96, 'date': datetime.datetime(2024, 11, 9, 12, 50, 52, 35538)}, {'home_team': 'Villarreal', 'away_team': 'Alaves', 'home_odds': 1.64, 'draw_odds': 4.06, 'away_odds': 5.19, 'date': datetime.datetime(2024, 11, 9, 12, 50, 52, 35538)}, {'home_team': 'Leganes', 'away_team': 'Sevilla', 'home_odds': 2.9, 'draw_odds': 3.06, 'away_odds': 2.65, 'date': datetime.datetime(2024, 11, 9, 12, 50, 52, 35538)}, {'home_team': 'Betis', 'away_team': 'Celta Vigo', 'home_odds': 1.99, 'draw_odds': 3.48, 'away_odds': 3.85, 'date': datetime.datetime(2024, 11, 10, 0, 0)}, {'home_team': 'Mallorca', 'away_team': 'Atl. Madrid', 'home_odds': 3.64, 'draw_odds': 2.99, 'a

In [44]:
def get_most_compatible_team(team):
    team_compatibility = teams

    team_compatibility["score"] = team_compatibility.apply(
        lambda x: fuzz.ratio(team, x["team"]),
        axis=1,
    )
    team_compatibility = team_compatibility.sort_values(
        by="score", ascending=False
    ).reset_index(drop=True)
    
    return team_compatibility.iloc[0]["team"]

In [45]:
data_model = []
for game in next_games:
    home_team_compatible = get_most_compatible_team(game["home_team"])
    print(f"\n{game['home_team']} --> {home_team_compatible}")
    game["home_team_translated"] = home_team_compatible
    
    away_team_compatible = get_most_compatible_team(game["away_team"])
    print(f"{game['away_team']} --> {away_team_compatible}")
    game["away_team_translated"] = away_team_compatible
    
    home_stats_dict = bf.get_team_previous_games_stats(
        game["home_team_translated"], season, game["date"], "H", min_games_played, min_games_played_at, season_games
    )
    if not home_stats_dict:
        continue

    away_stats_dict = bf.get_team_previous_games_stats(
        game["away_team_translated"], season, game["date"], "A", min_games_played, min_games_played_at, season_games
    )
    if not away_stats_dict:
        continue

    game_info_keys = [
        "date",
        "season",
        "home_team_translated",
        "away_team_translated",
        "home_odds",
        "away_odds",
        "draw_odds",
        "result",
        "home_score",
        "away_score",
    ]
    game_info_dict = {key: game.get(key) for key in game_info_keys}

    home_elo = teams_elo.get(game["home_team_translated"])
    away_elo = teams_elo.get(game["away_team_translated"])

    data_model.append({**home_stats_dict, **away_stats_dict, **game_info_dict, "home_elo": home_elo, "away_elo": away_elo})

data_df = pd.DataFrame(data_model)


Rayo Vallecano --> Rayo Vallecano
Las Palmas --> Las Palmas

Real Madrid --> Real Madrid
Osasuna --> Osasuna

Villarreal --> Villarreal
Alaves --> Alavés

Leganes --> Leganés
Sevilla --> Sevilla

Betis --> Betis
Celta Vigo --> Celta Vigo

Mallorca --> Mallorca
Atl. Madrid --> Atlético Madrid

Getafe --> Getafe
Girona --> Girona

Valladolid --> Valladolid
Ath Bilbao --> Athletic Club

Real Sociedad --> Real Sociedad
Barcelona --> Barcelona


In [46]:
display(data_df)

,home_points_pct,home_win_pct,home_draw_pct,home_loss_pct,home_points_pct_last_games,home_win_pct_last_games,home_draw_pct_last_games,home_loss_pct_last_games,home_home_win_pct,home_home_draw_pct,...,home_team_translated,away_team_translated,home_odds,away_odds,draw_odds,result,home_score,away_score,home_elo,away_elo
0,0.484848,0.363636,0.363636,0.272727,0.433333,0.3,0.4,0.3,0.400000,0.400000,...,Rayo Vallecano,Las Palmas,1.88,4.53,3.41,None,None,None,1485.92,1400.37
1,0.727273,0.636364,0.272727,0.090909,0.700000,0.6,0.3,0.1,0.833333,0.000000,...,Real Madrid,Osasuna,1.20,13.96,7.11,None,None,None,1825.93,1537.05
2,0.636364,0.545455,0.272727,0.181818,0.600000,0.5,0.3,0.2,0.400000,0.400000,...,Villarreal,Alavés,1.64,5.19,4.06,None,None,None,1625.92,1465.96
3,0.305556,0.166667,0.416667,0.416667,0.300000,0.2,0.3,0.5,0.400000,0.200000,...,Leganés,Sevilla,2.90,2.65,3.06,None,None,None,1472.39,1514.74
4,0.527778,0.416667,0.333333,0.250000,0.566667,0.5,0.2,0.3,0.666667,0.166667,...,Betis,Celta Vigo,1.99,3.85,3.48,None,None,None,1599.37,1498.85
5,0.500000,0.416667,0.250000,0.333333,0.566667,0.5,0.2,0.3,0.333333,0.500000,...,Mallorca,Atlético Madrid,3.64,2.29,2.99,None,None,None,1509.56,1714.89
6,0.277778,0.083333,0.583333,0.333333,0.266667,0.1,0.5,0.4,0.166667,0.833333,...,Getafe,Girona,2.44,3.16,3.09,None,None,None,1467.16,1620.58
7,0.222222,0.166667,0.166667,0.666667,0.166667,0.1,0.2,0.7,0.166667,0.333333,...,Valladolid,Athletic Club,4.55,1.87,3.39,None,None,None,1406.52,1629.97
8,0.416667,0.333333,0.250000,0.416667,0.400000,0.3,0.3,0.4,0.166667,0.166667,...,Real Sociedad,Barcelona,4.65,1.70,4.06,None,None,None,1596.61,1835.93


In [47]:
X = data_df[filtered_cols]

odds_cols = [
    "date",
    "season",
    "home_team_translated",
    "away_team_translated",
    "home_odds",
    "away_odds",
    "draw_odds",
    "home_elo",
    "away_elo",
]
odds_df = data_df[odds_cols]

for c in odds_cols:
    if "odds" in c:
        odds_df[c] = pd.to_numeric(odds_df[c], errors="coerce")

predictions = pipeline.predict(X)
probabilities = pipeline.predict_proba(X)

probs_test_df = pd.DataFrame(
    probabilities,
    index=data_df.index,
    columns=["away_probs", "draw_probs", "home_probs"],
)
preds_test_df = pd.DataFrame(predictions, index=data_df.index, columns=["pred"])
test_results_df = pd.concat([preds_test_df, probs_test_df, odds_df], axis=1)

test_results_df.dropna(subset=["home_odds"], inplace=True)
test_results_df = test_results_df[test_results_df["home_odds"] != " "]

test_results_df = test_results_df.astype(
    {"home_odds": float, "draw_odds": float, "away_odds": float}
)

In [50]:
today_bets = 0
for _, game in test_results_df.iterrows():
    bet_value = 1 # pf.get_bet_value_by_row(game, bankroll, strategy)
    odds, probs = pf.get_bet_odds_probs(game)
    
    bet_worth_it = pf.bet_worth_it(
        game["pred"],
        odds,
        1/probs,
        min_odds,
        bet_value
    )

    today_bets += 1

    bet_value = pf.get_bet_unit_value(odds, probs, bankroll, strategy, default_value, default_bankroll_pct)

    pred_str = "Draw"
    if game['pred'] == "H":
        pred_str = game['home_team_translated']
    elif game['pred'] == "A":
        pred_str = game['away_team_translated']

    if bet_worth_it:
        print(colored(f"{game['home_team_translated']} ({round(game['home_elo'], 2)}) x ({round(game['away_elo'], 2)}) {game['away_team_translated']}: ${bet_value} on {pred_str} @ {odds}", "green"))
    else:
        print(colored(f"{game['home_team_translated']} ({round(game['home_elo'], 2)}) x ({round(game['away_elo'], 2)}) {game['away_team_translated']}: ${bet_value} on {pred_str} @ {odds}", "red"))

if not today_bets:
    print("\nSorry, there are no bets for today.")

Rayo Vallecano (1485.92) x (1400.37) Las Palmas: $1 on Rayo Vallecano @ 1.88
Real Madrid (1825.93) x (1537.05) Osasuna: $1 on Real Madrid @ 1.2
Villarreal (1625.92) x (1465.96) Alavés: $1 on Villarreal @ 1.64
Leganés (1472.39) x (1514.74) Sevilla: $1 on Sevilla @ 2.65
Betis (1599.37) x (1498.85) Celta Vigo: $1 on Betis @ 1.99
Mallorca (1509.56) x (1714.89) Atlético Madrid: $1 on Atlético Madrid @ 2.29
Getafe (1467.16) x (1620.58) Girona: $1 on Draw @ 3.09
Valladolid (1406.52) x (1629.97) Athletic Club: $1 on Athletic Club @ 1.87
Real Sociedad (1596.61) x (1835.93) Barcelona: $1 on Barcelona @ 1.7
